# Reaction-diffusion
A simulation of reaction diffusion in Julia. Uses finite difference for calculating spatial derivatives and Euler for time integration. 
Periodic boundary conditions are enforced.

Results are rendered live using GLMakie package.

Ref: *Biological Pattern Formation : from Basic Mechanisms to Complex
Structures*, A. J. Koch and H. Meinhardt (1994) 

In [1]:
using Pkg
Pkg.activate(".")
using GLMakie

  Activating project at `~/CELESTE/PROJECTS/ReactionDiffusion`


WrapMatrix construct: has periodic boundaries; Wraps around.

In [4]:
struct WrapMatrix
    X::Matrix{Float64}
end
Base.size(x::WrapMatrix) = size(x.X)
Base.display(A::WrapMatrix) = Base.display(A.X)
Base.setindex!(X::WrapMatrix, x::Float64, i::Int64, j::Int64) = setindex!(X.X, x, i, j)
function Base.getindex(x::WrapMatrix, i::Int, j::Int)
    n,m=size(x)
    if (i > n)
        i = i % n
    elseif (i < 1)
        i = n + i % n
    end 
    if (j > m)
        j = j % m  
    elseif (j < 1)
        j = m + j % m
    end 
    x.X[i,j]
end

Calculates laplacian of a matrix

In [5]:
function Δ(X) 
    Δx² = 1. 
    m, n = size(X)
    X′ = WrapMatrix(X)
    ∇²X = zeros(m, n)
    for i in 1:m
        for j in 1:n
            ∇²X[i, j] += X′[i+1, j  ]
            ∇²X[i, j] += X′[i  , j+1] 
            ∇²X[i, j] += X′[i-1, j  ] 
            ∇²X[i, j] += X′[i  , j-1] 

            ∇²X[i, j] -= 4*X′[i, j] 
            ∇²X[i, j] /= Δx²
        end
    end
    return ∇²X
end

Δ (generic function with 1 method)

A reaction diffusion model with two reacting species

$$\frac{\partial a}{\partial t} = D_a\nabla^2a+\rho_a\frac{a^2s}{1+\kappa_a a^2}-\mu_a+\sigma_a\\
\frac{\partial s}{\partial t} = D_s\nabla^2s-\rho_s\frac{a^2s}{1+\kappa_a a^2}+\sigma_s
$$

In [7]:
function model1(a, b)
    Da = 0.005; Db = 0.2
    ρa = 0.01; ρb = 0.02
    μa = 0.01; μb = 0.02
    Ka = 0.25
    σa = 0.0; σb = 0.0

    ∇²a = Δ(a)
    ∇²b = Δ(b)

    ȧ = Da * ∇²a + 
        ρa * (a.^ 2 ./ (1 .+ Ka * a.^2)) ./ b - 
        μa*a .+ 
        σa
    
    ḃ = Db * ∇²b + 
        ρb * a.^2 -
        μb * b .+
        σb 

    return ȧ, ḃ
end

model1 (generic function with 1 method)

Time stepping using Euler

In [8]:
function update(a, b, model)
    Δt = 1.
    ȧ, ḃ = model(a, b)

    a .+= ȧ*Δt
    b .+= ḃ*Δt
end

update (generic function with 1 method)

Generate plot in GLMakie

In [9]:
function plotTrajectory(a, b, model, scene; iter=100, skipframes=1)
	heatmap!(scene, a, colormap=:plasma)
	begin
		for i = 1:iter
			update(a, b, model)
			scene[end][3][] = a
			sleep(1/1200)		
		end
	end
end

plotTrajectory (generic function with 1 method)

Initial conditions

In [10]:
gridx, gridy = 70, 70

Ai = rand(gridy, gridx);
Bi = rand(gridy, gridx);

In [11]:
A = copy(Ai)
B = copy(Bi);

Create and display a scene

In [12]:
scene = Scene()
heatmap!(scene, A, colormap=:plasma, show_axis=false)
scene

Live render each time step

In [13]:
plotTrajectory(A, B, model1, scene; iter=2000, skipframes=1)